<a href="https://colab.research.google.com/github/NoCodeProgram/deepLearning/blob/main/cnn/pooling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

input_tensor = torch.Tensor([[[[1, 2, 3, 4],
                             [5, 6, 7, 8],
                             [9, 10, 11, 12],
                             [13, 14, 15, 16]]]])
print(input_tensor.shape)
maxpool_layer = nn.MaxPool2d(kernel_size=2)
output_tensor = maxpool_layer(input_tensor)
# Step 5: Print the output tensor
print(output_tensor)


torch.Size([1, 1, 4, 4])
tensor([[[[ 6.,  8.],
          [14., 16.]]]])


In [ ]:

input_tensor = torch.Tensor([[[[1, 2, 3, 4],
                             [5, 6, 7, 8],
                             [9, 10, 11, 12],
                             [13, 14, 15, 16]]]])
avgpool_layer = nn.AvgPool2d(kernel_size=2)
output_tensor = avgpool_layer(input_tensor)
# Step 5: Print the output tensor
print(output_tensor)


tensor([[[[ 3.5000,  5.5000],
          [11.5000, 13.5000]]]])


In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt

if torch.backends.mps.is_available():
    my_device = torch.device('mps')
elif torch.cuda.is_available():
    my_device = torch.device('cuda')
else:
    my_device = torch.device('cpu')

print(my_device)

cpu


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Load CIFAR10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=4)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=4)


100%|██████████| 170498071/170498071 [00:03<00:00, 43750853.68it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified


In [ ]:


class ModernGAPCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1), # Stride 2 to reduce dimensions
            nn.ReLU(inplace=True),
        )

        self.global_avg_pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)  # Flatten the tensor before the fully connected layer
        x = self.classifier(x)
        return x

net = ModernGAPCNN(num_classes=10)


# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)


In [ ]:
net.to(my_device)
num_epochs = 100
for epoch in range(num_epochs):
    net.train()
    for batch_idx, (data, label) in enumerate(trainloader):
        data, label = data.to(my_device), label.to(my_device)
        scores = net(data)
        loss = criterion(scores, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    net.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for data, label in testloader:
            data, label = data.to(my_device), label.to(my_device)
            scores = net(data)
            loss = criterion(scores, label)
            val_loss += loss.item() * data.size(0)

            predicted = scores.argmax(dim=1)
            correct += predicted.eq(label).sum().item()

    val_loss /= len(testloader.dataset)
    val_accuracy = 100. * correct / len(testloader.dataset)

    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Epoch [1/100], Training Loss: 2.0488, Validation Loss: 1.8502, Validation Accuracy: 32.76%
Epoch [2/100], Training Loss: 1.7348, Validation Loss: 1.7183, Validation Accuracy: 37.32%
Epoch [3/100], Training Loss: 1.5089, Validation Loss: 1.6598, Validation Accuracy: 38.62%
Epoch [4/100], Training Loss: 1.6023, Validation Loss: 1.6354, Validation Accuracy: 39.50%
Epoch [5/100], Training Loss: 1.3982, Validation Loss: 1.6005, Validation Accuracy: 41.29%
Epoch [6/100], Training Loss: 1.3470, Validation Loss: 1.5722, Validation Accuracy: 42.95%
Epoch [7/100], Training Loss: 1.4415, Validation Loss: 1.5605, Validation Accuracy: 43.35%
Epoch [8/100], Training Loss: 1.3231, Validation Loss: 1.5241, Validation Accuracy: 44.90%
Epoch [9/100], Training Loss: 1.2867, Validation Loss: 1.5097, Validation Accuracy: 45.55%
Epoch [10/100], Training Loss: 1.2030, Validation Loss: 1.4861, Validation Accuracy: 46.83%
Epoch [11/100], Training Loss: 1.3143, Validation Loss: 1.4572, Validation Accuracy: 47.7